In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time
from torch.autograd import Variable

BATCH_SIZE = 128
NUM_EPOCHS = 10

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

In [3]:
import torch.optim as optim
import torch.nn.functional as F

class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
 
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
model = SimpleNet()

# TODO:define loss function and optimiter

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [4]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        images, labels = Variable(images), Variable(labels)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

train_correct = 0
test_correct = 0
for images, labels in tqdm(train_loader):
    images, labels = Variable(images), Variable(labels)
    outputs = model(images)
    pred = torch.argmax(outputs, 1)
    train_correct += (pred == labels).sum().float()
    
for images, labels in tqdm(test_loader):
    images, labels = Variable(images), Variable(labels)
    outputs = model(images)
    pred = torch.argmax(outputs, 1)
    test_correct += (pred == labels).sum().float()
    
train_accuracy = train_correct/(len(train_dataset))
test_accuracy = test_correct/(len(test_dataset))
    
print('Training accuracy: %0.2f%%' % (train_accuracy*100))
print('Testing accuracy: %0.2f%%' % (test_accuracy*100))
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset

100%|██████████| 78/78 [00:01<00:00, 69.98it/s]

Training accuracy: 95.71%
Testing accuracy: 95.94%
